In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df=pd.read_csv('LoanStats_2017Q2.csv')

In [38]:
x = df['loan_status']
x.value_counts()

Current               99850
Fully Paid             3896
In Grace Period         932
Late (31-120 days)      436
Late (16-30 days)       312
Charged Off              25
Name: loan_status, dtype: int64

#filling certain columns that i think are NA due to not having a result. will sort this further down
df[['mths_since_recent_bc_dlq','mths_since_last_major_derog','mths_since_recent_revol_delinq','mths_since_last_delinq']] = df[['mths_since_recent_bc_dlq','mths_since_last_major_derog','mths_since_recent_revol_delinq','mths_since_last_delinq']].fillna(-1)


In [39]:
def binner(columnName):
    # Get boolean values indicating null
    mths_null = np.isnan(columnName)

    # Negate the booleans with  ~ and use them to select all the non-null values, leaving null values out
    valid = columnName[~mths_null]
    
    
    mean = valid.mean()

    # Fill the NaNs with -1
    columnName = columnName.fillna(-1)
    
    columnName = columnName.apply(lambda x: clean_up_2(x, mean))

            
    # Use np.digitize to bin the values, with NaNs in bin 0, others in 1...5
    return columnName

In [40]:
# I am assuming the NaN's in this column are representitive of not having delinq.
# before filling, the mean was roughly 35 for all columns so i have split the results around 35. 
def clean_up_2(x,mean):
    
   
    
    if x <0:
        return 0
    
    elif x<mean:
        return 2
    else:
        return 1
    

In [41]:

change_list = ['mths_since_last_delinq','mths_since_last_major_derog','mths_since_recent_revol_delinq','mths_since_recent_bc_dlq']

for col in change_list:
    df[col] = binner(df[col])

In [42]:
risk =['Late (16-30 days)','Late (31-120 days)','Charged Off','In Grace Period']

no_risk =['Current','Fully Paid']

def binary_maker(x):
    if x in risk:
        return 0
    else:
        return 1
    
df['loan_status']=df['loan_status'].apply(lambda x:binary_maker(x))    

In [43]:
#setting max view and looking at dataframe
pd.set_option('display.max_columns', None)


In [44]:
df['annual_inc_joint'].fillna(0,inplace=True)

df['total_inc']=df['annual_inc_joint']+df['annual_inc']

In [45]:
#creating a mask to find percents nan over 60 percent
mask = df.columns[((df.isnull().sum()/105451)*100>=60)]
df[mask].head()

,id,member_id,url,desc,mths_since_last_record,dti_joint,verification_status_joint,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
#dropping the NaN columns over 60 percent
df.drop(mask,axis=1,inplace=True)

In [47]:
nulls = df.columns[df.isnull().any()]

df[nulls].head(5)

,emp_title,emp_length,dti,revol_util,last_pymnt_d,next_pymnt_d,last_credit_pull_d,mths_since_rcnt_il,il_util,all_util,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mths_since_recent_bc,mths_since_recent_inq,num_tl_120dpd_2m,percent_bc_gt_75
0,teacher,10.0,25.61,105.80%,Aug-17,Sep-17,Aug-17,26.0,69.0,94.0,26051.0,94.0,96.1,26.0,2.0,8.0,0.0,100.0
1,Registered Nurse,9.0,31.58,86.70%,Aug-17,Sep-17,Aug-17,13.0,57.0,71.0,5855.0,1421.0,94.8,118.0,21.0,13.0,0.0,100.0
2,Dealer,7.0,12.19,12.40%,Aug-17,Sep-17,Aug-17,26.0,99.0,46.0,5862.0,48443.0,12.4,153.0,33.0,22.0,0.0,60.0
3,QA Manager,1.0,31.39,98%,Aug-17,Sep-17,Aug-17,13.0,51.0,98.0,112860.0,10514.0,100.0,46.0,8.0,0.0,NaN,100.0
4,Physician Assistant,0.0,18.34,48.60%,Aug-17,Sep-17,Aug-17,10.0,58.0,56.0,21193.0,9527.0,49.6,132.0,67.0,3.0,0.0,28.6


In [48]:
#these columns represent too much work or not enough information. Emp title would be very useful but would take a long time to process.
#may come back for this one. 
df.drop(['emp_title','out_prncp_inv', 'total_pymnt_inv','last_pymnt_amnt','last_credit_pull_d','annual_inc_joint'],inplace=True,axis=1)

In [49]:
#I'm assuming that for all people who do not have an employee length that they are unemployed.
df['emp_length'].fillna(value=-1,inplace=True)


# i am sorting by arbitrary employment lengths
def clean_up(x):
    if x < 0:
        return 0
    elif x <=1:
        return 1
    elif x<=5:
        return 2
    else:
        return 3
    
# this function returns 0 for unemployed, 1 for 0-1 years 2 for 1-5 and 5+ years employment.

df['emp_length'] = df['emp_length'].apply(lambda x: clean_up(x)).astype('category')


In [50]:
df['recovery'] = np.where((df.recoveries != 0.00), 1, 0)

In [51]:
#dropping some more useless lines
df.drop(['issue_d','title','zip_code','earliest_cr_line','addr_state','funded_amnt',
         'pymnt_plan','policy_code','collection_recovery_fee','recoveries'],inplace=True,axis=1)

In [52]:


#
cln_list = ['last_pymnt_d','next_pymnt_d']


df[cln_list]=df[cln_list].fillna('None')

In [53]:
import re


def sub(text):
    text=str(text)
    x=re.sub(r'%?','',text)
    return x



df['int_rate']= df['int_rate'].apply(lambda x: sub(x))

df['int_rate']=df['int_rate'].astype('float')

df['int_rate']=df['int_rate']

df['revol_util']= df['revol_util'].apply(lambda x: sub(x))

df['revol_util']=df['revol_util'].astype('float')


In [54]:
nulls = df.columns[df.isnull().any()]



In [55]:
# filling all values with 0 at the moment
df[nulls]=df[nulls].fillna(df[nulls].mean())

In [56]:
y = df['loan_status']
X=df.drop('loan_status',axis=1)



X=pd.get_dummies(X)



In [57]:
X.shape

(105451, 162)

In [58]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,classification_report,f1_score,recall_score,precision_score


X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=1)

In [59]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.
C:\Users\JimGill\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\JimGill\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\JimGill\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\JimGill\anaconda3\lib\site-packages\tensorflow\python\framework\d

In [60]:
rus = RandomUnderSampler(sampling_strategy={1:4000})
ros = RandomOverSampler()



In [61]:
#X_train,y_train=rus.fit_sample(X_train,y_train)
#X_train,y_train=ros.fit_sample(X_train,y_train)

In [62]:
X_train.shape

(73815, 162)

In [63]:
import xgboost as xgb

In [64]:
#d_train = xgb.DMatrix(data=X_train.values,label=y_train.values)
#d_test = xgb.DMatrix(data=X_test.values)

In [65]:
positive= (len(y_train[y_train==0]))
neg =(len(y_train))-positive
class_imbalance = neg/len(y_train)

class_imbalance

0.9838379733116576

In [66]:


classifier = xgb.XGBClassifier(objective='binary:logistic',random_state =1,n_estimators=1500)
classifier.set_params(**{'reg_lambda': 0,
 'reg_alpha': 0,
 'n_estimators': 1000,
 'min_child_weight': 7,
 'max_depth': 5,
 'learning_rate': 0.05,
 'colsample_bytree': 1,
 'colsample_bynode': 0.9,
 'colsample_bylevel': 1})
classifier.fit(X_train,y_train)





XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=0.9, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=7, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [67]:
preds=classifier.predict(X_test)
confusion_matrix(y_test,preds)

array([[  398,   114],
       [   81, 31043]], dtype=int64)

In [68]:
probs = classifier.predict_proba(X_test)

In [69]:
def predict(array):
    new_preds = np.zeros(len(array[:,1]))
    for i in range(len(array[:,1])):
        if array[i,1] >0.999:
            new_preds[i] = 1
        else:
            new_preds[i] =0
    return new_preds
    


In [70]:
new_pred=predict(probs)

In [71]:
confusion_matrix(y_test,new_pred)

array([[  472,    40],
       [ 6790, 24334]], dtype=int64)

In [72]:
print(recall_score(y_test,new_pred,average=None))
print(precision_score(y_test,new_pred,average=None))

[0.921875   0.78184038]
[0.06499587 0.99835891]


In [74]:
classifier.feature_importances_

array([0.00181285, 0.00304103, 0.00134063, 0.0037852 , 0.00409268,
       0.00286018, 0.00300644, 0.00377573, 0.00292333, 0.00488913,
       0.00336898, 0.00187453, 0.00303395, 0.00365447, 0.00305151,
       0.00487928, 0.00524288, 0.02172082, 0.00304002, 0.01711326,
       0.        , 0.00299305, 0.        , 0.00341339, 0.0027719 ,
       0.00287776, 0.00235189, 0.00313546, 0.00246753, 0.00333099,
       0.00299416, 0.00379853, 0.0041438 , 0.00201211, 0.00353614,
       0.00311491, 0.00291689, 0.00226789, 0.00309124, 0.00376684,
       0.00318333, 0.00286718, 0.00301133, 0.00329102, 0.        ,
       0.        , 0.00273357, 0.00284558, 0.0032242 , 0.00300527,
       0.00344817, 0.00331139, 0.00244768, 0.00302772, 0.00360871,
       0.00225308, 0.00298482, 0.00319162, 0.0033627 , 0.00277993,
       0.00285762, 0.00370422, 0.0028129 , 0.00336974, 0.00232959,
       0.00536504, 0.        , 0.        , 0.00294252, 0.00335377,
       0.00326127, 0.00245504, 0.00163083, 0.00276648, 0.00320

In [ ]:


params={
    'n_estimators':[100,500,1000],
    'max_depth':[3,4,5,6],
    'learning_rate': [0.01,0.05,0.1],
    'colsample_bytree':[0.9,1],
    'min_child_weight': [1,3,5,7],
    'colsample_bylevel':[0.9,1],
    'colsample_bynode':[0.9,1],
    'reg_alpha':[0,1,5],
    'reg_lambda':[0,1,5] 
    
}
classifier_2 = xgb.XGBClassifier(objective='binary:logistic')
rand = RandomizedSearchCV(estimator=classifier_2,param_distributions=params,n_iter=40,cv=4,scoring='accuracy',n_jobs=-1,verbose=2)



In [ ]:
rand.fit(X_train,y_train)

In [ ]:
rand.best_score_

In [ ]:
rand.best_params_

In [ ]:
#df[df['loan_status']=='In Grace Period'].head(20)

#df[df['loan_status'] =='Late (31-120 days)']

#df[df['loan_status']=='Late (16-30 days)']          

#df[df['loan_status']=='Charged Off']       